In [1]:
from pyspark.sql import SparkSession
from random import randint
from pyspark.sql.types import IntegerType, StructField, StructType
import pyspark.sql.functions as F
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from tensorflow.keras.optimizers import RMSprop 
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable

In [3]:
spark = SparkSession.builder.appName('SparkTitanic.com').getOrCreate()

Reading the data

In [4]:
train_df = spark.read.option("inferSchema", True)\
    .option("header", True)\
    .csv('./input/titanic/train.csv')
    #for production: '../input/titanic/train.csv'

Cleaning the data

In [5]:
train_df = train_df.select("Sex", "Age", "Survived")

In [6]:
train_df = train_df.where(train_df["Age"].isNotNull())

In [7]:
train_df = train_df.where(train_df["Sex"].isNotNull())

In [8]:
train_df = train_df.where(train_df["Survived"].isNotNull())

In [9]:
train_df = train_df.withColumn("Sex", F.when(F.col("Sex")==F.lit("male"), 0.).otherwise(1.) )

In [10]:
x_train = list(map(lambda x: list(x), train_df.select("Sex", "Age").collect()))
y_train = list(map(lambda x: list(x)[0], train_df.select("Survived").collect()))


In [11]:
del(train_df)

In [12]:
x_train = np.array([[x[0], x[1]*0.01] for x in x_train], dtype="float32")
#normalization

In [13]:
y_train = to_categorical(y_train,2)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.30, random_state=40)

Training

In [14]:
model = Sequential()

In [15]:
model.add(Dense(128, input_dim = 2, activation='sigmoid'))

In [16]:
model.add(Dense(256, activation='relu'))

In [17]:
model.add(BatchNormalization(momentum=0.8))

In [18]:
model.add(Dense(1024, activation='tanh'))

In [19]:
model.add(Dense(2048, activation='relu'))

In [20]:
model.add(BatchNormalization(momentum=0.8))

In [21]:
model.add(Dense(1024, activation='tanh'))

In [22]:
model.add(BatchNormalization(momentum=0.8))

In [23]:
model.add(Dense(256, activation='sigmoid'))

In [24]:
model.add(BatchNormalization(momentum=0.8))

In [25]:
model.add(Dense(128, activation='tanh'))

In [26]:
model.add(BatchNormalization(momentum=0.8))

In [27]:
model.add(Dense(2,activation='softmax'))

In [28]:
model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

In [29]:
model.fit(x_train, y_train, batch_size=100, epochs=300, shuffle=True, verbose=0)

In [ ]:
model.evaluate(x_test, y_test, batch_size=20)

In [30]:
del(x_train)
del(y_train)

In [ ]:
del(x_test)
del(y_test)

Prediction

In [31]:
def prediction(row):
    if row.Sex == 'female':
        Sex = 1.
    elif row.Sex == 'male':
        Sex = 0.
    else:
        Sex = randint(0,1)*1.
    if row.Age == None:
        Age = randint(0,100)*0.01
    else:
        Age = row.Age*0.01

    p = model.predict([(Sex, Age)])[0]

    return int(np.argmax(p))

In [32]:
test_df = spark.read.option("inferSchema", True)\
    .option("header", True)\
    .csv('./input/titanic/test.csv')
    #for production: '../input/titanic/test.csv'

In [33]:
output_df = spark.createDataFrame(spark.sparkContext.parallelize([]), StructType([
    StructField('PassengerId', IntegerType(), True),
    StructField('Survived', IntegerType(), True)
]))

In [34]:
for row in test_df.collect():
    output_df = output_df.union( spark.sparkContext.parallelize([(row.PassengerId, prediction(row) )]).toDF(['PassengerId', 'Survived']) )

Output

In [35]:
output_df.toPandas().to_csv('./submission.csv', index=False)